# Agents with Human-in-the-Loop

We have an email assistant that uses a router to triage emails and then passes the email to the agent for response generation. We've also evaluated it. But do we fully *trust* it to manage our inbox autonomously? For such a sensitive task, human-in-the-loop (HITL) is important! Here we'll show how to add a human-in-the-loop to our email assistant so that we can review specific tool calls. 

![overview-img](img/overview_hitl.png)



First, let's preview it. Let's run a local deployment of our email assistant with HITL from `src/email_assistant_hitl.ts`  As before, run `pnpm agent`, select `hitlEmailAssistant` in Studio, and submit the e-mail:

The graph will *now pause* before specific tools calls, such as `write_email`.

![overview-img](img/hitl_schematic.png)

You can see the specific tool call that triggered the interrupt in Studio. 

![studio-img](img/studio-interrupt.png)

We'll use a custom interface to handle these interrupts called [Agent Inbox](https://dev.agentinbox.ai/). This interface is a nice way to edit, approve, ignore, or provide feedback on specific actions taken by LangGraph agents.  If you go to [dev.agentinbox.ai](https://dev.agentinbox.ai/), you can add the graph url:
   * Graph name: the name from the `langgraph.json` file (`hitlEmailAssistant`)
   * Graph URL: `http://localhost:2024/`

All interrupted threads run will be visible: 

![agent-inbox-img](img/agent-inbox.png)

## Adding HITL to our email assistant

Now that we've seen interrupt in Studio with Agent Inbox, let's add HITL to our email assistant. 

We can start with tools, just as we did before. We will also add some zod schemas necessary for the tools.

But now, we'll add a few new tools including `Question` that will allow the assist to ask the user a question. We can answer questions via HITL! 

In [1]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";
import { StateGraph, START, END, interrupt, Command } from "@langchain/langgraph";
import { initChatModel } from "langchain/chat_models/universal";
import { AIMessage, BaseMessage, HumanMessage } from "@langchain/core/messages";
import { Messages, addMessages } from "@langchain/langgraph";
import "@langchain/langgraph/zod";

// Define the Zod schemas for the email assistant states
const MessagesState = z.object({
    messages: z
        .custom<BaseMessage[]>()
        .default(() => [])
        .langgraph.reducer<Messages>((left, right) => addMessages(left, right)),
});

const BaseEmailAgentState = MessagesState.extend({
    email_input: z.any(),
    classification_decision: z
        .enum(["ignore", "respond", "notify"])
        .nullable()
        .default(null),
});

const EmailAgentHITLState = MessagesState.extend({
    email_input: z.any(),
    classification_decision: z
        .enum(["ignore", "respond", "notify", "error"])
        .nullable()
        .default(null),
});

// Export the inferred types from the Zod schemas
type BaseEmailAgentStateType = z.infer<typeof BaseEmailAgentState>;
type EmailAgentHITLStateType = z.infer<typeof EmailAgentHITLState>;

// Email tool with correct properties
const emailSchema = z.object({
    recipient: z.string().describe("Email address of the recipient"),
    subject: z.string().describe("Clear and concise subject line for the email"),
    content: z.string().describe("Main body text of the email"),
});

const writeEmail = tool(async ({
    recipient,
    subject,
    content,
}: z.infer<typeof emailSchema>) => {
    return `Email draft created:
To: ${recipient}
Subject: ${subject}
${content}
[Draft saved. Ready to send or edit further.]`;
}, {
    name: "write_email",
    description:
        "Write an email draft based on provided information. Use this when the user wants to compose a new email message.",
    schema: emailSchema,
});

// Calendar tool with correct properties
const scheduleMeetingSchema = z.object({
    title: z.string().describe("Meeting title"),
    attendees: z.array(z.string()).describe("List of attendees' emails"),
    startTime: z.string().describe("Meeting start time in ISO format"),
    endTime: z.string().describe("Meeting end time in ISO format"),
    description: z.string().optional().describe("Meeting description"),
});

const scheduleMeeting = tool(async (args: z.infer<typeof scheduleMeetingSchema>) => {
    const { title, attendees, startTime, endTime, description } = args;
    // Mock implementation
    return `Meeting "${title}" scheduled from ${startTime} to ${endTime} with ${attendees.length} attendees`;
}, {
    name: "schedule_meeting",
    description: "Schedule a meeting on the calendar",
    schema: scheduleMeetingSchema,
});

const availabilitySchema = z.object({
    startTime: z.string().describe("Start time in ISO format"),
    endTime: z.string().describe("End time in ISO format"),
});

const checkCalendarAvailability = tool(async (args: z.infer<typeof availabilitySchema>) => {
    const { startTime, endTime } = args;
    // Mock implementation
    return `Time slot from ${startTime} to ${endTime} is available`;
}, {
    name: "check_calendar_availability",
    description: "Check calendar availability for a specified time range",
    schema: availabilitySchema,
},
);

const questionTool = tool(
    async ({ content }: { content: string }) => {
        return `The user will see and can answer this question: ${content}`;
    },
    {
        name: "question",
        description: "Ask the user a follow-up question",
        schema: z.object({
            content: z.string().describe("The question to ask the user"),
        }),
    },
);

// Define the doneSchema
const doneSchema = z.object({
    content: z.string().optional().describe("Optional completion message"),
});

const Done = tool(async () => {
    return "Task completed successfully. No further actions required.";
}, {
    name: "Done",
    description:
        "Signal that you've completed the current task and no further actions are needed.",
    schema: doneSchema,
});

const tools = [
    writeEmail,
    scheduleMeeting,
    checkCalendarAvailability,
    questionTool,
    Done,
];

const toolsByName = Object.fromEntries(tools.map((t) => [t.name, t]));

// LLM setup
const llm = initChatModel("openai:gpt-4.1", { temperature: 0.0 });

// Using async/await and promise chaining for models with structured outputs
const setupModels = async () => {
    const baseRouterModel = await initChatModel("openai:gpt-4.1", { temperature: 0.0 });
    const baseToolsModel = await initChatModel("openai:gpt-4.1", { temperature: 0.0 });
    const llmRouter = baseRouterModel.withStructuredOutput(RouterSchema);
    const llmWithTools = baseToolsModel.bindTools(tools, { toolChoice: "required" });
    return { llmRouter, llmWithTools };
};

const RouterSchema = z.object({
    reasoning: z
        .string()
        .describe("Step-by-step reasoning behind the classification"),
    classification: z
        .enum(["ignore", "respond", "notify"])
        .describe(
            "The classification of an email: 'ignore' for irrelevant emails, " +
            "'notify' for important information that doesn't need a response, " +
            "'respond' for emails that need a reply",
        ),
});

// Input schema (once)
const StateInput = z.object({
    email_input: z.any(),
});

// For simplified state access in functions
interface State {
    email_input: any;
    classification_decision?: string;
    messages: BaseMessage[];
}

In [2]:
// Tool descriptions for HITL workflow
const HITL_TOOLS_PROMPT = `
1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day, start_time) - Schedule calendar meetings where preferred_day is a datetime object
3. check_calendar_availability(day) - Check available time slots for a given day
4. Question(content) - Ask the user any follow-up questions
5. Done - E-mail has been sent
`;
console.log(HITL_TOOLS_PROMPT);


1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day, start_time) - Schedule calendar meetings where preferred_day is a datetime object
3. check_calendar_availability(day) - Check available time slots for a given day
4. Question(content) - Ask the user any follow-up questions
5. Done - E-mail has been sent



#### Triage node

We define a TypeScript function with our triage routing logic, just as we did before.

But, if the classification is `notify`, we want to interrupt the graph to allow the user to review the email! So we go to a new node, `triageRouter`.

In [3]:
function parseEmail(email: EmailData): {
  author: string;
  to: string;
  subject: string;
  emailThread: string;
} {
  try {
      // Extract key information from email data
      const author = email.from_email;
      const to = email.to_email;
      const subject = email.subject;
      const emailThread = email.page_content;

      return { author, to, subject, emailThread };
  } catch (error) {
      console.error("Error parsing email:", error);
      throw new Error("Failed to parse email");
  }
}

const defaultTriageInstructions = `
Emails that are not worth responding to:
- Marketing newsletters and promotional emails
- Spam or suspicious emails
- CC'd on FYI threads with no direct questions

There are also other things that should be known about, but don't require an email response. For these, you should notify (using the \`notify\` response). Examples of this include:
- Team member out sick or on vacation
- Build system notifications or deployments
- Project status updates without action items
- Important company announcements
- FYI emails that contain relevant information for current projects
- HR Department deadline reminders
- Subscription status / renewal reminders
- GitHub notifications

Emails that are worth responding to:
- Direct questions from team members requiring expertise
- Meeting requests requiring confirmation
- Critical bug reports related to team's projects
- Requests from management requiring acknowledgment
- Client inquiries about project status or features
- Technical questions about documentation, code, or APIs (especially questions about missing endpoints or features)
- Personal reminders related to family (wife / daughter)
- Personal reminder related to self-care (doctor appointments, etc)
`;

///////PROMPTS 
const defaultBackground = `
I'm Lance, a software engineer at LangChain.
`;
// Agentic workflow triage user prompt
const triageUserPrompt = `
Please determine how to handle the below email thread:

From: {author}
To: {to}
Subject: {subject}
{email_thread}`;




const triageSystemPrompt = `
<Role>
Your role is to triage incoming emails based upon instructs and background information below.
</Role>

<Background>
{background}. 
</Background>

<Instructions>
Categorize each email into one of three categories:
1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that worth notification but doesn't require a response
3. RESPOND - Emails that need a direct response
Classify the below email into one of these categories.
</Instructions>

<Rules>
{triage_instructions}
</Rules>
`;

function formatEmailMarkdown(
  subject: string,
  author: string,
  to: string,
  emailThread: string,
): string {
  return `## Email: ${subject}
**From**: ${author}
**To**: ${to}
${emailThread}`;
}

type EmailData = {
  id: string;
  thread_id: string;
  from_email: string;
  subject: string;
  page_content: string;
  send_time: string;
  to_email: string;
};

async function triageRouter(state: State) {
  const { author, to, subject, emailThread } = parseEmail(state.email_input);

  const emailMarkdown = formatEmailMarkdown(subject, author, to, emailThread);
  // Fix the string template replacement issue
  const userPrompt = triageUserPrompt
      .replace("{author}", author)
      .replace("{to}", to)
      .replace("{subject}", subject)
      .replace("{email_thread}", emailThread);
  // Fix the string template replacement issue
  const systemPrompt = triageSystemPrompt
      .replace("{background}", defaultBackground)
      .replace("{triage_instructions}", defaultTriageInstructions);

  // Await the model to be initialized first
  const model = await initChatModel("openai:gpt-4.1", { temperature: 0.0 });
  const routerModel = model.withStructuredOutput(RouterSchema);

  // Await the invoke result
  const result = await routerModel.invoke([
      { role: "system", content: systemPrompt },
      { role: "user", content: userPrompt },
  ]);

  const classification = result.classification;

  if (classification === "respond") {
      console.log("📧 Classification: RESPOND - This email requires a response");
      return new Command({
          goto: "response_agent",
          update: {
              classification_decision: result.classification,
              messages: [
                  {
                      role: "user",
                      content: `Respond to the email: ${emailMarkdown}`,
                  },
              ],
          },
      });
  } else if (classification === "ignore") {
      console.log("🚫 Classification: IGNORE - This email can be safely ignored");
      return new Command({
          goto: END,
          update: { classification_decision: classification },
      });
  } else if (classification === "notify") {
      console.log("🔔 Classification: NOTIFY - This email contains important information");
      return new Command({
          goto: "triage_interrupt_handler",
          update: { classification_decision: classification },
      });
  } else {
      return new Command({
          goto: END,
          update: { classification_decision: classification },
      });
  }
}

But, remember now we want to interrupt the graph at the `notify` classification to allow the user to review the email. 

![overview-img](img/hitl_schematic.png)

For this, we simply add a new node, `triageInterruptHandler`, that will: 

1. Show the classification to the user if it is `notify`: We'll pass an object to the interrupt that contains our classification. 
2. Allow the user to respond to the decision: We'll design the code to handle what we will get back from Agent Inbox. 

As you can see [here](https://github.com/langchain-ai/agent-inbox?tab=readme-ov-file#what-do-the-fields-mean) (note: Agent Inbox itself might be a separate system, ensure its API aligns with your LangGraph JS interrupt structure), we format our interrupt with specific fields so that it can be viewed in Agent Inbox:

* `action_request`: The action and arguments for the interrupt with `action` (the action name) and `args` (the tool call arguments). This is rendered in the Agent Inbox as the main header for the interrupt event.
* `config`: Configures which interaction types are allowed, and specific UI elements for each. 
* `description`: Should be detailed, and may be markdown. This will be rendered in the Agent Inbox as the description

In [4]:
function triageInterruptHandler(state) {
  const { author, to, subject, emailThread } = parseEmail(state.email_input);
  const emailMarkdown = formatEmailMarkdown(subject, author, to, emailThread);

  const messages = [
      {
          role: "user",
          content: `Email to notify user about: ${emailMarkdown}`,
      },
  ];

  const request = {
      action_request: {
          action: `Email Assistant: ${state.classification_decision}`,
          args: {},
      },
      config: {
          allow_ignore: true,
          allow_respond: true,
          allow_edit: false,
          allow_accept: false,
      },
      description: emailMarkdown,
  };

  const response = interrupt([request])[0];

  // Declare routing destination variable
  let routeTo;

  if (response.type === "response") {
      const userInput = response.args;
      messages.push({
          role: "user",
          content: `User wants to reply to the email. Use this feedback to respond: ${userInput}`,
      });
      routeTo = "response_agent";
  } else if (response.type === "ignore") {
      routeTo = END;
  } else {
      throw new Error(`Invalid response: ${JSON.stringify(response)}`);
  }

  return new Command({
      goto: routeTo,
      update: { messages },
  });
}

The `llmCall` node is the same as before:

In [5]:
import z from "zod";
import { initChatModel } from "langchain/chat_models/universal";

// Define the router schema using Zod
const RouterSchema = z.object({
    reasoning: z.string().describe("Step-by-step reasoning behind the classification."),
    classification: z.enum(["ignore", "respond", "notify"]).describe(
        "The classification of an email: 'ignore' for irrelevant emails, " +
        "'notify' for important information that doesn't need a response, " +
        "'respond' for emails that need a reply"
    )
});

// Default calendar preferences
const defaultCalPreferences = `
30 minute meetings are preferred, but 15 minute meetings are also acceptable.
`;

// Tool descriptions for HITL workflow
const HITL_TOOLS_PROMPT = `
1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day, start_time) - Schedule calendar meetings where preferred_day is a datetime object
3. check_calendar_availability(day) - Check available time slots for a given day
4. Question(content) - Ask the user any follow-up questions
5. Done - E-mail has been sent
`;

// Agentic workflow with HITL prompt
const agentSystemPromptHitl = `
<Role>
You are a top-notch executive assistant who cares about helping your executive perform as well as possible.
</Role>

<Tools>
You have access to the following tools to help manage communications and schedule:
{tools_prompt}
</Tools>

<Instructions>
When handling emails, follow these steps:
1. Carefully analyze the email content and purpose
2. IMPORTANT --- always call a tool and call one tool at a time until the task is complete: 
3. If you need more information to complete the task, use the Question tool to ask a follow-up question to the user 
4. For responding to the email, draft a response email with the write_email tool
5. For meeting requests, use the check_calendar_availability tool to find open time slots
6. To schedule a meeting, use the schedule_meeting tool with a datetime object for the preferred_day parameter
   - Today's date is ${new Date().toISOString().split("T")[0]} - use this for scheduling meetings accurately
7. If you scheduled a meeting, then draft a short response email using the write_email tool
8. After using the write_email tool, the task is complete
9. If you have sent the email, then use the Done tool to indicate that the task is complete
</Instructions>

<Background>
{background}
</Background>

<Response Preferences>
{response_preferences}
</Response Preferences>

<Calendar Preferences>
{cal_preferences}
</Calendar Preferences>
`;


const defaultResponsePreferences = `
Use professional and concise language. If the e-mail mentions a deadline, make sure to explicitly acknowledge and reference the deadline in your response.

When responding to technical questions that require investigation:
- Clearly state whether you will investigate or who you will ask
- Provide an estimated timeline for when you'll have more information or complete the task

When responding to event or conference invitations:
- Always acknowledge any mentioned deadlines (particularly registration deadlines)
- If workshops or specific topics are mentioned, ask for more specific details about them
- If discounts (group or early bird) are mentioned, explicitly request information about them
- Don't commit 

When responding to collaboration or project-related requests:
- Acknowledge any existing work or materials mentioned (drafts, slides, documents, etc.)
- Explicitly mention reviewing these materials before or during the meeting
- When scheduling meetings, clearly state the specific day, date, and time proposed

When responding to meeting scheduling requests:
- If times are proposed, verify calendar availability for all time slots mentioned in the original email and then commit to one of the proposed times based on your availability by scheduling the meeting. Or, say you can't make it at the time proposed.
- If no times are proposed, then check your calendar for availability and propose multiple time options when available instead of selecting just one.
- Mention the meeting duration in your response to confirm you've noted it correctly.
- Reference the meeting's purpose in your response.
`;


const llm = await initChatModel(
    "openai:gpt-4o",
    { temperature: 0.0 }
);


const llmWithTools = llm.bindTools(tools, { toolChoice: "required" });

async function llmCall(state) {
    /**
     * LLM decides whether to call a tool or not
     * This is the main decision-making node that generates responses or tool calls
     */
    const messages = [...state.messages];
    const systemPromptContent = agentSystemPromptHitl
        .replace("{tools_prompt}", HITL_TOOLS_PROMPT)
        .replace("{background}", defaultBackground)
        .replace("{response_preferences}", defaultResponsePreferences)
        .replace("{cal_preferences}", defaultCalPreferences);

    // Run the LLM with the messages
    const response = await llmWithTools.invoke([
        { role: "system", content: systemPromptContent },
        ...messages,
    ]);

    // Return messages in the format expected by the graph
    return {
        messages: response,
    };
}

The `interrupt_handler` is the core HITL component of our response agent. 

Its job is to examine the tool calls that the LLM wants to make and determine which ones need human review before execution. Here's how it works:

1. **Tool Selection**: The handler maintains a list of "HITL tools" that require human approval:
   - `write_email`: Since sending emails has significant external impact
   - `schedule_meeting`: Since scheduling meetings affects calendars
   - `Question`: Since asking users questions requires direct interaction

2. **Direct Execution**: Tools not in the HITL list (like `check_calendar_availability`) are executed immediately without interruption. This allows low-risk operations to proceed automatically.

3. **Context Preparation**: For tools requiring review, the handler:
   - Retrieves the original email for context
   - Formats the tool call details for clear display
   - Configures which interaction types are allowed for each tool type

4. **Interrupt Creation**: The handler creates a structured interrupt request with:
   - The action name and arguments
   - Configuration for allowed interaction types
   - A description that includes both the original email and the proposed action

5. **Response Processing**: After the interrupt, the handler processes the human response:
   - **Accept**: Executes the tool with original arguments
   - **Edit**: Updates the tool call with edited arguments and then executes
   - **Ignore**: Cancels the tool execution
   - **Response**: Records feedback without execution

This handler ensures humans have oversight of all significant actions while allowing routine operations to proceed automatically. 

The ability to edit tool arguments (like email content or meeting details) gives users precise control over the assistant's actions.

We can visualize the overall flow: 

![overview-img](img/HITL_flow.png)

Agent Inbox will allow us review each tool call. As an example, with `write_email` you can fully edit the email content in a rich text editor, allowing for precise control over the final message. Agent Inbox returns a list of dicts with a single key `type` that can be `accept`, `edit`, `ignore`, or `response`.   

![agent-inbox-img](img/agent-inbox-draft.png)

In [6]:
import { z } from "zod";
import {
    StateGraph,
    START,
    END,
    interrupt,
    Command
} from "@langchain/langgraph";
import { AIMessage, BaseMessage, HumanMessage, ToolMessage } from "@langchain/core/messages";
import { Messages, addMessages } from "@langchain/langgraph";
import "@langchain/langgraph/zod";
import { ToolCall } from "@langchain/core/messages/tool";

// Helper for type checking
const hasToolCalls = (
    message: BaseMessage
): message is AIMessage & { tool_calls: ToolCall[] } => {
    return (
        message.getType() === "ai" &&
        "tool_calls" in message &&
        Array.isArray((message as any).tool_calls) &&
        (message as any).tool_calls.length > 0
    );
};

// Define proper state schema
const MessagesState = z.object({
    messages: z
        .custom<BaseMessage[]>()
        .default(() => [])
        .langgraph.reducer<Messages>((left, right) => addMessages(left, right)),
});

// Extend MessagesState for our email agent
const EmailAgentState = MessagesState.extend({
    email_input: z.any(),
    classification_decision: z
        .enum(["ignore", "respond", "notify", "error"])
        .nullable()
        .default(null),
});

// Define types from schemas
type EmailAgentStateType = z.infer<typeof EmailAgentState>;

// Input schema
const StateInputSchema = z.object({
    email_input: z.any(),
});
type StateInputType = z.infer<typeof StateInputSchema>;

// Create the interrupt handler node
async function interruptHandler(
    state: EmailAgentStateType
): Promise<Command> {
    // Store messages to be returned
    const result: BaseMessage[] = [];
    // Default goto is llm_call
    let goto: typeof END | "llm_call" = "llm_call";

    // Get the last message
    const lastMessage = state.messages[state.messages.length - 1];

    // Exit early if there are no tool calls
    if (!hasToolCalls(lastMessage)) {
        return new Command({
            goto,
            update: { messages: [] },
        });
    }

    // Process one tool call at a time
    for (const toolCall of lastMessage.tool_calls) {
        const hitlTools = ["write_email", "schedule_meeting", "Question"];

        if (!hitlTools.includes(toolCall.name)) {
            const tool = toolsByName[toolCall.name];
            const observation = await tool.invoke(toolCall.args);
            result.push(new ToolMessage({
                content: observation,
                tool_call_id: toolCall.id
            }));
            continue;
        }

        const emailInput = state.email_input;
        const { author, to, subject, emailThread } = parseEmail(emailInput);
        const originalEmailMarkdown = formatEmailMarkdown(subject, author, to, emailThread);
        function formatForDisplay(toolCall: ToolCall): string {
            // Initialize empty display
            let display = "";

            // Add tool call information based on tool type
            switch (toolCall.name) {
                case "write_email":
                    display += `# Email Draft
    
    **To**: ${toolCall.args.to}
    **Subject**: ${toolCall.args.subject}
    
    ${toolCall.args.content}
    `;
                    break;

                case "schedule_meeting":
                    display += `# Calendar Invite
    
    **Meeting**: ${toolCall.args.subject}
    **Attendees**: ${toolCall.args.attendees?.join(", ")}
    **Duration**: ${toolCall.args.duration_minutes} minutes
    **Day**: ${toolCall.args.preferred_day}
    `;
                    break;

                case "question":
                    // Special formatting for questions to make them clear
                    display += `# Question for User
    
    ${toolCall.args.content}
    `;
                    break;

                default:
                    // Generic format for other tools
                    display += `# Tool Call: ${toolCall.name}
    
    Arguments:
    ${JSON.stringify(toolCall.args, null, 2)}
    `;
            }

            return display;
        }
        const toolDisplay = formatForDisplay(toolCall);
        const description = originalEmailMarkdown + toolDisplay;

        let config;
        if (toolCall.name === "write_email" || toolCall.name === "schedule_meeting") {
            config = {
                allow_ignore: true,
                allow_respond: true,
                allow_edit: true,
                allow_accept: true,
            };
        } else if (toolCall.name === "Question") {
            config = {
                allow_ignore: true,
                allow_respond: true,
                allow_edit: false,
                allow_accept: false,
            };
        } else {
            throw new Error(`Invalid tool call: ${toolCall.name}`);
        }

        const request = {
            action_request: {
                action: toolCall.name,
                args: toolCall.args,
            },
            config,
            description,
        };

        const response = interrupt([request])[0];

        if (response.type === "accept") {
            const tool = toolsByName[toolCall.name];
            const observation = await tool.invoke(toolCall.args);
            result.push(new ToolMessage({
                content: observation,
                tool_call_id: toolCall.id
            }));
        } else if (response.type === "edit") {
            const tool = toolsByName[toolCall.name];
            const editedArgs = response.args;
            const observation = await tool.invoke(editedArgs);
            result.push(new ToolMessage({
                content: observation,
                tool_call_id: toolCall.id
            }));
        } else if (response.type === "ignore") {
            result.push(new ToolMessage({
                content: `User ignored this ${toolCall.name} draft. Ignore this and end the workflow.`,
                tool_call_id: toolCall.id
            }));
            goto = END;
        } else if (response.type === "response") {
            const userFeedback = response.args;
            result.push(new ToolMessage({
                content: `User gave feedback, which can we incorporate into the ${toolCall.name}. Feedback: ${userFeedback}`,
                tool_call_id: toolCall.id
            }));
        } else {
            throw new Error(`Invalid response: ${JSON.stringify(response)}`);
        }
    }

    return new Command({
        goto,
        update: { messages: result }
    });
}

// Create shouldContinue function for conditional edges
function shouldContinue(state: EmailAgentStateType) {
    const messages = state.messages;
    if (!messages || messages.length === 0) return END;

    const lastMessage = messages[messages.length - 1];

    if (hasToolCalls(lastMessage)) {
        // Check if any tool call is the "Done" tool
        if (lastMessage.tool_calls.some(toolCall => toolCall.name === "Done")) {
            return END;
        }
        return "interrupt_handler";
    }

    return END;
}

// Build the agent workflow using the graph builder already defined
const agentBuilder = new StateGraph(EmailAgentState)
    .addNode("llm_call", llmCall)
    .addNode("interrupt_handler", interruptHandler)
    .addEdge(START, "llm_call")
    .addConditionalEdges("llm_call", shouldContinue)
    .addEdge("interrupt_handler", "llm_call");

Now, we can test some of the HITL patterns. 

## Accept the `write_email` tool call

In [7]:
import { v4 as uuidv4 } from "uuid";
import { interrupt } from "@langchain/langgraph";

// Email to respond to
const emailInputRespond = {
    to: "Lance Martin <lance@company.com>",
    author: "Project Manager <pm@client.com>",
    subject: "Tax season let's schedule call",
    email_thread: `Lance,

It's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.

Are you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.

Regards,
Project Manager`
};

// Compile the agent subgraph first
const responseAgent = agentBuilder.compile();

// Now build the overall workflow graph
const overallWorkflow = new StateGraph(EmailAgentState)
    .addNode("triage_router", triageRouter)
    .addNode("triage_interrupt_handler", triageInterruptHandler)
    .addNode("response_agent", responseAgent)
    .addEdge(START, "triage_router")
    .addConditionalEdges(
        "triage_router",
        (state) => {
            const classification = state.classification_decision;
            if (classification === "respond") {
                return "response_agent";
            } else if (classification === "notify") {
                return "triage_interrupt_handler";
            } else {
                return END;
            }
        }
    )
    .addConditionalEdges(
        "triage_interrupt_handler",
        (state) => {
            const messages = state.messages;
            if (!messages || messages.length === 0) return END;
            const lastMessage = messages[messages.length - 1];
            // Use getType() for message type check
            if (
                lastMessage.getType() === "human" &&
                typeof lastMessage.content === "string" &&
                lastMessage.content.includes("User wants to reply to the email")
            ) {
                return "response_agent";
            }
            return END;
        }
    )
    .addEdge("response_agent", END);

// Compile the graph 
const graph = overallWorkflow.compile();
const threadId1 = uuidv4();
const threadConfig1 = { configurable: { thread_id: threadId1 } };

// Run the graph until a tool call that we choose to interrupt
console.log("Running the graph until the first interrupt...");
for await (const chunk of await graph.stream({ email_input: emailInputRespond }, threadConfig1)) {
    console.log(chunk)
    if ("__interrupt__" in chunk) {
        const interruptObject = chunk.__interrupt__[0];
        console.log("\nINTERRUPT OBJECT:");
        console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
    }
}

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response
{
  triage_router: { classification_decision: 'respond', messages: [ [Object] ] }
}
{
  response_agent: {
    messages: [
      HumanMessage {
        "id": "0f71b6fe-54c8-41ac-8552-790740de4540",
        "content": "Respond to the email: ## Email: Tax season let's schedule call\n**From**: undefined\n**To**: undefined\nundefined",
        "additional_kwargs": {},
        "response_metadata": {}
      },
      AIMessage {
        "id": "chatcmpl-BWVQ7iyjQ7p2n54hK39LLPLYrbcKH",
        "content": "",
        "additional_kwargs": {
          "tool_calls": [
            {
              "id": "call_oGwsrNfD15TqYMQaSqtD9wtL",
              "type": "function",
              "function": "[Object]"
            }
          ]
        },
        "response_metadata": {
          "tokenUsage": {
            "promptTokens": 968,
            "completionTokens": 63,
            "totalTokens": 1031


What happened? In our agent loop, we added a simple LangGraph [interrupt](https://langchain-ai.github.io/langgraphjs/concepts/human_in_the_loop/) (see also [how to wait for user input](https://langchain-ai.github.io/langgraphjs/how-tos/human_in_the_loop/wait-user-input/)), which allows us to pause execution of an agent at a specific point in the code. The interrupt allowed us to pass through the tool call for the user to review!


```typescript
import { interrupt } from "@langchain/langgraph";

const feedback = interrupt.invoke(myToolCall); 
```

You can see the `action` (tool call name) and `args` (tool call arguments) that we want to interrupt. Now, how do we handle the interrupt? This is where the `Command` interface comes in . [The `Command` object in LangGraph has several powerful capabilities](https://langchain-ai.github.io/langgraphjs/how-tos/command/). Previously we saw it used to direct the flow of the graph: 
- `goto`: Specifies which node to route to next
- `update`: Modifies the state before continuing execution

Here, we'll use a similar concept to resume the graph from the interrupted state:
- `resume` : Provides the value to return from the interrupt call

We can return whatever value our graph is designed to handle. In our case, the graph is designed to handle a list of objects with a single key `type` that can be `accept`, `edit`, `ignore`, or `response`. So, we can simply pass `{ type: "accept" }` to the resume mechanism in order to tell the graph that we accept the tool call.

In [8]:
import { v4 as uuidv4 } from "uuid";
import { Command } from "@langchain/langgraph";

// Run the graph until a tool call that we choose to interrupt
console.log("Running the graph until the first interrupt...");
for await (const chunk of await graph.stream({ email_input: emailInputRespond }, threadConfig1)) {
    if ("__interrupt__" in chunk) {
        const interruptObject = chunk.__interrupt__[0];
        console.log("\nINTERRUPT OBJECT:");
        console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);

        // Example: Simulate user accepting the tool call
        await graph.invoke(
            new Command({ resume: [{ type: "accept" }] }),
            threadConfig1
        );
        break; // Stop after first interrupt for demonstration
    }
}

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response


In [9]:
(async () => {
  const state = await graph.getState(threadConfig1);
  const messages = state.values.messages;
  if (Array.isArray(messages)) {
      for (const m of messages) {
          console.log(m);
      }
  } else {
      console.log("No messages found in state:", messages);
  }
})();

Promise {
  <pending>,
  [Symbol(async_id_symbol)]: 35611,
  [Symbol(trigger_async_id_symbol)]: 35601,
  [Symbol(kResourceStore)]: undefined
}


UnhandledPromiseRejection: GraphValueError: No checkpointer set
    at CompiledStateGraph.getState (/Users/dylan/Desktop/agents-from-scratch-ts/node_modules/.pnpm/@langchain+langgraph@0.2.71_@langchain+core@0.3.50_openai@4.96.2_zod@3.24.3___zod-to-json-schema@3.24.5_zod@3.24.3_/node_modules/@langchain/langgraph/dist/pregel/index.cjs:654:19)
    at evalmachine.<anonymous>:4:39
    at evalmachine.<anonymous>:14:3
    at evalmachine.<anonymous>:16:3
    at sigintHandlersWrap (node:vm:280:12)
    at Script.runInThisContext (node:vm:135:14)
    at Object.runInThisContext (node:vm:317:38)
    at Object.execute (/Users/dylan/Desktop/agents-from-scratch-ts/node_modules/.pnpm/tslab@1.0.22/node_modules/tslab/dist/executor.js:160:38)
    at JupyterHandlerImpl.handleExecuteImpl (/Users/dylan/Desktop/agents-from-scratch-ts/node_modules/.pnpm/tslab@1.0.22/node_modules/tslab/dist/jupyter.js:250:38)
    at /Users/dylan/Desktop/agents-from-scratch-ts/node_modules/.pnpm/tslab@1.0.22/node_modules/tslab/d



### Edit `write_email` and `schedule_meeting`

This test demonstrates how human modification works in the HITL flow:
1. We start with the same tax planning email as before
2. The agent proposes a meeting with the same parameters
3. This time, the user EDITS the meeting proposal to change:
   - Duration from 45 to 30 minutes
   - Meeting subject is made more concise
4. The agent adapts to these changes when drafting the email
5. The user further EDITS the email to be shorter and less formal
6. The workflow completes with both modifications incorporated

This scenario showcases one of the most powerful aspects of HITL: 

* Users can make precise modifications to agent actions before they are executed, ensuring the final outcome matches their preferences without having to handle all the details themselves.

In [10]:
// Email to respond to
const emailInputRespond = {
  to: "Lance Martin <lance@company.com>",
  author: "Project Manager <pm@client.com>",
  subject: "Tax season let's schedule call",
  email_thread: `Lance,

It's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.

Are you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.

Regards,
Project Manager`
};

// Compile the graph with new thread
const graph = overallWorkflow.compile();
const threadId2 = uuidv4();
const threadConfig2 = { configurable: { thread_id: threadId2 } };

// Run the graph until the first interrupt 
// will be classified as "respond" and the agent will create a write_email tool call
console.log("Running the graph until the first interrupt...");
for await (const chunk of await graph.stream({ email_input: emailInputRespond }, threadConfig2)) {
  if ("__interrupt__" in chunk) {
      const interruptObject = chunk.__interrupt__[0];
      console.log("\nINTERRUPT OBJECT:");
      console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
  }
}

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response


Edit the `schedule_meeting` tool call

When the agent proposes the initial meeting schedule, we now simulate the user making modifications through the edit functionality. This demonstrates how the `edit` response type works:

1. The user receives the same meeting proposal as in the previous test
2. Instead of accepting, they modify the parameters:
   - Reducing duration from 45 to 30 minutes
   - Keeping the same day and time
3. The `edit` response includes the complete set of modified arguments
4. The interrupt handler replaces the original tool arguments with these edited ones
5. The tool is executed with the user's modifications

This shows how edit capability gives users precise control over agent actions while still letting the agent handle the execution details.

In [11]:
import { MemorySaver } from "@langchain/langgraph";

const memory = new MemorySaver();
const graph = overallWorkflow.compile({ checkpointer: memory });

console.log("\nSimulating user editing the schedule_meeting tool call...");
const editedScheduleArgs = {
    attendees: ["pm@client.com", "lance@company.com"],
    subject: "Tax Planning Discussion",
    duration_minutes: 30,
    preferred_day: "2025-05-06",
    start_time: 14,
};

for await (const chunk of await graph.stream(
    new Command({ resume: [{ type: "edit", args: { args: editedScheduleArgs } }] }),
    threadConfig2
)) {
    if (chunk.response_agent?.messages) {
        // Print the most recent message from the response agent
        console.log(chunk.response_agent.messages[chunk.response_agent.messages.length - 1]);
    }
    if ("__interrupt__" in chunk) {
        const interruptObject = chunk.__interrupt__[0];
        console.log("\nINTERRUPT OBJECT:");
        console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
    }
}


Simulating user editing the schedule_meeting tool call...


Edit the `write_email` tool call

After accepting the modified meeting schedule, the agent drafts an email reflecting the 30-minute duration. Now we demonstrate how editing works with email content:

1. The agent has adapted its email to mention the shorter 30-minute duration
2. We simulate the user wanting an even more significant change to the email:
   - Completely rewriting the content to be shorter and less formal
   - Changing the meeting day mentioned in the email (showing how users can correct agent mistakes)
   - Requesting confirmation rather than stating the meeting as definite
3. The `edit` response contains the complete new email content
4. The tool arguments are updated with this edited content
5. The email is sent with the user's preferred wording

This example shows the power of HITL for complex communication tasks - the agent handles the structure and initial content, while humans can refine tone, style, and substance.

In [12]:
console.log("\nSimulating user editing the write_email tool call...");
const editedEmailArgs = {
    to: "pm@client.com",
    subject: "Re: Tax season let's schedule call",
    content: `Hello Project Manager,

Thank you for reaching out about tax planning. I scheduled a 30-minute call next Thursday at 3:00 PM. Would that work for you?

Best regards,
Lance Martin`
};

for await (const chunk of await graph.stream(
    new Command({ resume: [{ type: "edit", args: { args: editedEmailArgs } }] }),
    threadConfig2
)) {
    if (chunk.response_agent?.messages) {
        console.log(chunk.response_agent.messages[chunk.response_agent.messages.length - 1]);
    }
    if ("__interrupt__" in chunk) {
        const interruptObject = chunk.__interrupt__[0];
        console.log("\nINTERRUPT OBJECT:");
        console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
    }
}


Simulating user editing the write_email tool call...


Look at the full message history, and see trace, to view the edited tool calls:

https://smith.langchain.com/public/21769510-d57a-41e4-b5c7-0ddb23c237d8/r

In [13]:
(async () => {
  const state = await graph.getState(threadConfig2);
  for (const m of state.values.messages) {
    console.log(m);
  }
})();

Promise {
  <pending>,
  [Symbol(async_id_symbol)]: 42559,
  [Symbol(trigger_async_id_symbol)]: 42549,
  [Symbol(kResourceStore)]: undefined
}


### Respond (with feedback) `write_email`, `schedule_meeting`, and `question`

This test set demonstrates the "response" capability - providing feedback without editing or accepting:

1. First, we test feedback for meeting scheduling:
   - The user provides specific preferences (30 minutes instead of 45, and afternoon meetings)
   - The agent incorporates this feedback into a revised proposal
   - The user then accepts the revised meeting schedule

2. Second, we test feedback for email drafting:
   - The user requests a shorter, less formal email with a specific closing statement
   - The agent completely rewrites the email according to this guidance
   - The user accepts the new draft

3. Lastly, we test feedback for questions:
   - For the brunch invitation, the user answers the question with additional context
   - The agent uses this information to draft an appropriate email response
   - The workflow proceeds with the user's input integrated

The "response" capability bridges the gap between acceptance and editing - users can guide the agent without having to write the full content themselves. This is especially powerful for:
- Adjusting tone and style
- Adding context the agent missed
- Redirecting the agent's approach
- Answering questions in a way that shapes the next steps

In [14]:
import { v4 as uuidv4 } from "uuid";
import { MemorySaver } from "@langchain/langgraph";

// Respond - Meeting Request Email
const emailInputRespond = {
    to: "Lance Martin <lance@company.com>",
    author: "Project Manager <pm@client.com>",
    subject: "Tax season let's schedule call",
    email_thread: `Lance,

It's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.

Are you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.

Regards,
Project Manager`
};

// Compile the graph
const checkpointer = new MemorySaver();
const graph = overallWorkflow.compile({ checkpointer });
const threadId5 = uuidv4();
const threadConfig5 = { configurable: { thread_id: threadId5 } };

// Run the graph until the first interrupt
console.log("Running the graph until the first interrupt...");
for await (const chunk of await graph.stream({ email_input: emailInputRespond }, threadConfig5)) {
    if ("__interrupt__" in chunk) {
        const interruptObject = chunk.__interrupt__[0];
        console.log("\nINTERRUPT OBJECT:");
        console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
    }
}

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response


Provide feedback for the `schedule_meeting` tool call

Now we explore the feedback capability for meeting scheduling:

1. The agent proposes the standard 45-minute meeting on Tuesday at 2:00 PM
2. Instead of accepting or editing, we provide feedback in natural language
3. Our feedback specifies two preferences:
   - Shorter meeting (30 minutes instead of 45)
   - Preference for afternoon meetings (after 2pm)
4. The agent receives this feedback through the `response` type
5. The interrupt handler adds this feedback as a message to the state
6. The agent processes this feedback and generates a new tool call incorporating these preferences

Unlike direct editing, which requires specifying the entire set of parameters, feedback allows users to express their preferences conversationally. The agent must then interpret this feedback and apply it appropriately to create a revised proposal.

In [15]:
console.log("\nSimulating user providing feedback for the schedule_meeting tool call...");
for await (const chunk of await graph.stream(
    new Command({
        resume: [
            {
                type: "response",
                args: "Please schedule this for 30 minutes instead of 45 minutes, and I prefer afternoon meetings after 2pm."
            }
        ]
    }),
    threadConfig2
)) {
    if ("__interrupt__" in chunk) {
        const interruptObject = chunk.__interrupt__[0];
        console.log("\nINTERRUPT OBJECT:");
        console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
    }
}


Simulating user providing feedback for the schedule_meeting tool call...


Accept the `schedule_meeting` tool call after providing feedback

In [16]:
console.log("\nSimulating user providing feedback for the schedule_meeting tool call...");
for await (const chunk of await graph.stream(
    new Command({
        resume: [
            {
                type: "response",
                args: "Please schedule this for 30 minutes instead of 45 minutes, and I prefer afternoon meetings after 2pm."
            }
        ]
    }),
    threadConfig2
)) {
    if ("__interrupt__" in chunk) {
        const interruptObject = chunk.__interrupt__[0];
        console.log("\nINTERRUPT OBJECT:");
        console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
    }
}


Simulating user providing feedback for the schedule_meeting tool call...


Now provide feedback for the `write_email` tool call

After accepting the revised meeting schedule, the agent drafts an email. We now test feedback for email content:

1. The agent's email is relatively formal and detailed
2. We provide stylistic feedback requesting:
   - A shorter, more concise email
   - A less formal tone
   - A specific closing statement about looking forward to the meeting
3. The agent processes this feedback to completely rewrite the email
4. The new draft is much shorter, more casual, and includes the requested closing

This demonstrates the power of natural language feedback for content creation:
- Users don't need to rewrite the entire email themselves
- They can provide high-level guidance on style, tone, and content
- The agent handles the actual writing based on this guidance
- The result better matches user preferences while preserving the essential information

The message history shows both the original and revised emails, clearly showing how the feedback was incorporated.

In [17]:
console.log("\nSimulating user providing feedback for the write_email tool call...");
for await (const chunk of await graph.stream(
    new Command({
        resume: [
            {
                type: "response",
                args: "Shorter and less formal. Include a closing statement about looking forward to the meeting!"
            }
        ]
    }),
    threadConfig2
)) {
    if (chunk.response_agent?.messages) {
        console.log(chunk.response_agent.messages[chunk.response_agent.messages.length - 1]);
    }
    if ("__interrupt__" in chunk) {
        const interruptObject = chunk.__interrupt__[0];
        console.log("\nINTERRUPT OBJECT:");
        console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
    }
}


Simulating user providing feedback for the write_email tool call...


Accept the `write_email` tool call after providing feedback

In [18]:
// Simulate user accepting the write_email tool call after providing feedback
console.log(`\nSimulating user accepting the tool call...`);

const stream = await graph.stream(
    new Command({ resume: [{ type: "accept" }] }),
    threadConfig2
);

for await (const chunk of stream) {
    if ("__interrupt__" in chunk) {
        const interruptObject = chunk.__interrupt__[0];
        console.log("\nINTERRUPT OBJECT:");
        console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
    }
}


Simulating user accepting the tool call...


Look at the full message history, and see the trace:

https://smith.langchain.com/public/57006770-6bb3-4e40-b990-143c373ebe60/r

We can see that user feedback in incorporated into the tool calls.  

In [19]:
const stateAfterFeedback = await graph.getState(threadConfig2);
for (const m of stateAfterFeedback.values.messages) {
  console.log(m);
}

Now let's try an email that calls the `Question` tool to provide feedback

Finally, we test how feedback works with the `Question` tool:

1. For the brunch invitation email, the agent asks about preferred day and time
2. Instead of ignoring, we provide a substantive response with additional context:
   - Confirming we want to invite the people mentioned
   - Noting we need to check which weekend works best
   - Adding information about needing a reservation
3. The agent uses this information to:
   - Draft a comprehensive email response incorporating all our feedback
   - Notice we didn't provide a specific day/time, so it suggests checking the calendar
   - Include the detail about making a reservation
4. The complete email reflects both the original request and our additional guidance

This demonstrates how question responses can shape the entire workflow:
- Questions let the agent gather missing information
- User responses can include both direct answers and additional context
- The agent integrates all this information into its next actions
- The final outcome reflects the collaborative intelligence of both human and AI

In [20]:
// Respond to a Question tool call (e.g., brunch invitation)
const emailInputBrunch = {
  to: "Lance Martin <lance@company.com>",
  author: "Partner <partner@home.com>",
  subject: "Dinner?",
  email_thread: "Hey, do you want italian or indian tonight?"
};

const threadId3 = uuidv4();
const threadConfig3 = { configurable: { thread_id: threadId3 } };

console.log("Running the graph until the first interrupt...");

// Await the stream if necessary (if graph.stream returns a Promise)
const stream = await graph.stream({ email_input: emailInputBrunch }, threadConfig3);

for await (const chunk of stream) {
  if ("__interrupt__" in chunk) {
      const interruptObject = chunk.__interrupt__[0];
      console.log("\nINTERRUPT OBJECT:");
      console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
  }
}

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response


Provide feedback for the `Question` tool call

In [21]:
// Simulate user providing feedback for the Question tool call
console.log("\nSimulating user providing feedback for the Question tool call...");

const stream = await graph.stream(
    new Command({
        resume: [
            {
                type: "response",
                args: "Let's do indian."
            }
        ]
    }),
    threadConfig3 // <-- pass directly, not wrapped in { config: ... }
);

for await (const chunk of stream) {
    if ("__interrupt__" in chunk) {
        const interruptObject = chunk.__interrupt__[0];
        console.log("\nINTERRUPT OBJECT:");
        console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
    }
}


Simulating user providing feedback for the Question tool call...


Accept the `write_email` tool call

In [22]:
// Simulate user accepting the write_email tool call after Question
console.log("\nSimulating user accepting the write_email tool call...");

const stream = await graph.stream(
    new Command({ resume: [{ type: "accept" }] }),
    threadConfig3 // <-- pass directly, not wrapped in { config: ... }
);

for await (const chunk of stream) {
    if (chunk.response_agent?.messages) {
        console.log(chunk.response_agent.messages[chunk.response_agent.messages.length - 1]);
    }
    if ("__interrupt__" in chunk) {
        const interruptObject = chunk.__interrupt__[0];
        console.log("\nINTERRUPT OBJECT:");
        console.log(`Action Request: ${JSON.stringify(interruptObject.value[0].action_request)}`);
    }
}


Simulating user accepting the write_email tool call...


Look at the full message history, and see the trace:

https://smith.langchain.com/public/f4c727c3-b1d9-47a5-b3d0-3451619db8a2/r

We can see that user feedback in incorporated into the email response.

In [23]:
const state = await graph.getState(threadConfig3);
state.values.messages.forEach(m => m.prettyPrint());

TypeError: m.prettyPrint is not a function
    at evalmachine.<anonymous>:5:59
    at Array.forEach (<anonymous>)
    at evalmachine.<anonymous>:5:44
    at async Object.execute (/Users/dylan/Desktop/agents-from-scratch-ts/node_modules/.pnpm/tslab@1.0.22/node_modules/tslab/dist/executor.js:173:17)
    at async JupyterHandlerImpl.handleExecuteImpl (/Users/dylan/Desktop/agents-from-scratch-ts/node_modules/.pnpm/tslab@1.0.22/node_modules/tslab/dist/jupyter.js:250:18)
    at async JupyterHandlerImpl.handleExecute (/Users/dylan/Desktop/agents-from-scratch-ts/node_modules/.pnpm/tslab@1.0.22/node_modules/tslab/dist/jupyter.js:208:21)
    at async ZmqServer.handleExecute (/Users/dylan/Desktop/agents-from-scratch-ts/node_modules/.pnpm/tslab@1.0.22/node_modules/tslab/dist/jupyter.js:406:25)
    at async ZmqServer.handleShellMessage (/Users/dylan/Desktop/agents-from-scratch-ts/node_modules/.pnpm/tslab@1.0.22/node_modules/tslab/dist/jupyter.js:351:21)
